Order Setup

In [1]:
import asyncio
import websockets
import json
import pandas as pd
import pprint
import nest_asyncio
nest_asyncio.apply()

async def call_api(msg):
    async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
        ###############
       # Before sending a message, make sure that your connection
       # is authenticated (use public/auth call before) 
       ###############
        await websocket.send(msg)
        while websocket.open:
            response = await websocket.recv()
            json_par = json.loads(response)
            print(json_par)
            return(json_par)
        
auth_msg = {
  "jsonrpc" : "2.0",
  "id" : 9929,
  "method" : "public/auth",
  "params" : {
    "grant_type" : "client_credentials",
    "client_id" : "YOUR_ID_HERE",
    "client_secret" : "YOUR_API_SECRET_HERE"
  }}

Perpetual Order

In [ ]:
response = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(auth_msg)))

msg = {
  "jsonrpc" : "2.0",
  "id" : 4122,
  "method" : "private/cancel_all_by_instrument",
  "params" : {
    "instrument_name" : "BTC-PERPETUAL",
    "type" : "all"
  }
}

response = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(msg)))

Futures Order

In [ ]:
msg = {
  "jsonrpc" : "2.0",
  "id" : 4122,
  "method" : "private/cancel_all_by_instrument",
  "params" : {
    "instrument_name" : "BTC-24SEP21",
    "type" : "all"
  }
}

response = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(msg)))

Options Order

In [ ]:
msg = {
  "jsonrpc" : "2.0",
  "id" : 4122,
  "method" : "private/cancel_all_by_instrument",
  "params" : {
    "instrument_name" : "ETH-19MAR21",
    "type" : "all"
  }
}

response = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(msg)))

Order Example 1

In [ ]:
#Order Foundation
import asyncio
import websockets
import json
import pandas as pd
from time import sleep
import pprint
import hmac
import hashlib
from datetime import datetime
import nest_asyncio
nest_asyncio.apply()

async def call_api(msg):
    async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
        await websocket.send(msg)
        while websocket.open:
            response = await websocket.recv()
            json_par = json.loads(response)
            #print(json_par)
            return(json_par)
        
clientId = "ID_HERE"
clientSecret = "SECRET_HERE"
timestamp = round(datetime.now().timestamp() * 1000)
nonce = "123"
data = ""

signature = hmac.new(
    bytes(clientSecret, "latin-1"),
    msg=bytes('{}\n{}\n{}'.format(timestamp, nonce, data), "latin-1"),
    digestmod=hashlib.sha256).hexdigest().lower()        
        
auth_msg = {
  "jsonrpc" : "2.0",
  "id" : 42,
  "method" : "public/auth",
  "params" : {
    "grant_type" : "client_signature",
    "client_id" : clientId,
    "timestamp" : timestamp,
    "signature" : signature,
    "nonce" : nonce,
    "data" : data
  }
}

price_msg = {
  "jsonrpc" : "2.0",
  "id" : 42,
  "method" : "public/ticker",
  "params" : {
    "instrument_name" : "BTC-PERPETUAL"
  }
}


order_msg = {
  "jsonrpc" : "2.0",
  "id" : 42,
  "method" : "private/buy",
  "params" : {
    "instrument_name" : "ETH-PERPETUAL",
    "amount" : 5,
    "type" : "market",
    "label" : "algoorder"
  }
}

check_order = {
  "jsonrpc" : "2.0",
  "id" : 42,
  "method" : "private/get_order_state",
  "params" : {
    "order_id" : "ETH-331562"
  }
}

#Main order loop
while True:
    try:
        btc = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(price_msg)))
    except exception as e:
        print("Unable to obtain BTC price")
    
    if btc['result']['best_ask_price'] < 58800.0:
        print("The price mark was not reached.")
        sleep(60)
        continue
    
    elif btc['result']['best_ask_price'] >= 58800.0:
        try:
            auth = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(auth_msg)))
            pprint.pprint(auth)
        except exception as e:
            print('There was an authentication error')
        
        try:
            order = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(order_msg)))
            #print(order)
        except exception as e:
            print("Error occurred while placing order")
        
        sleep(2)
        
        try:
            order_check = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(check_order)))
            pprint.pprint(order_check)
        except excpetion as e:
            print("Error checking order")
        
        if order_check['result']['order_state'] == 'filled' or order_check['result']['order_state'] == "open":
            print ('Order placed at {}'.format(pd.Timestamp.now()))
            break
            
        else:
            print('Order was canceled {}'.format(pd.Timestamp.now()))
            break

Order Example 2

In [ ]:
clientId = "ID_HERE"
clientSecret = "SECRET_HERE"
timestamp = round(datetime.now().timestamp() * 1000)
nonce = "123"
data = ""

signature = hmac.new(
    bytes(clientSecret, "latin-1"),
    msg=bytes('{}\n{}\n{}'.format(timestamp, nonce, data), "latin-1"),
    digestmod=hashlib.sha256).hexdigest().lower()        
        
auth_msg = {
  "jsonrpc" : "2.0",
  "id" : 42,
  "method" : "public/auth",
  "params" : {
    "grant_type" : "client_signature",
    "client_id" : clientId,
    "timestamp" : timestamp,
    "signature" : signature,
    "nonce" : nonce,
    "data" : data
  }
}

btc_price_msg = {
  "jsonrpc" : "2.0",
  "id" : 42,
  "method" : "public/ticker",
  "params" : {
    "instrument_name" : "BTC-PERPETUAL"
  }
}

order_msg = {
  "jsonrpc" : "2.0",
  "id" : 42,
  "method" : "private/buy",
  "params" : {
    "instrument_name" : "ETH-PERPETUAL",
    "amount" : 5,
    "type" : "market",
    "label" : "algoorder"
  }
}

check_order = {
  "jsonrpc" : "2.0",
  "id" : 42,
  "method" : "private/get_order_state",
  "params" : {
    "order_id" : "ETH-331562"
  }
}

while True:
    try:
        btc_old = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(btc_price_msg)))
    except exception as e:
        print("Unable to obtain BTC price")
    
    sleep(300)
    
    try:
        btc_new = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(btc_price_msg)))
    except exception as e:
        print("Unable to obtain BTC price")
    
    percent = (btc_new['result']['best_ask_price'] - btc_old['result']['best_ask_price'] * 100) / btc_old['result']['best_ask_price']
    
    if percent < 5:
        print("The requirement was not fulfilled.")
        sleep(0.1)
        continue
    
    elif percent >= 5:
        try:
            auth = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(auth_msg)))
            pprint.pprint(auth)
        except exception as e:
            print('There was an authentication error')
        
        try:
            order = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(order_msg)))
            #print(order)
        except exception as e:
            print("Error occurred while placing order")
        
        sleep(2)
        
        try:
            order_check = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(check_order)))
            pprint.pprint(order_check)
        except excpetion as e:
            print("Error checking order")
        
        if order_check['result']['order_state'] == 'filled' or order_check['result']['order_state'] == "open":
            print ('Order placed at {}'.format(pd.Timestamp.now()))
            break
            
        else:
            print('Order was canceled {}'.format(pd.Timestamp.now()))
            break